## Tick History Request- Sanctions

 Based on "RTH Python Code Samples":
 <br>
 https://developers.refinitiv.com/en/api-catalog/refinitiv-tick-history/refinitiv-tick-history-rth-rest-api/download
 <br>
 
### Introduction to the Approach:
* Obtain valid token
* Request all user packages - select packageId of interest (for example, RRUS)
* Request package deliveries -select packageDeliveryId of interest
* Download file(s) of interest
* Optionally, download all files per packageId


### Import Required Modules and Define Parameters

In [5]:
import requests
import json
import shutil
import time
import urllib3
import gzip
import copy
import asyncio

reqStart = "https://selectapi.datascope"  #endpoint URL start

filePath = ".\\downloads\\"  #Location to save downloaded files

###  Read Credentials from File

In [6]:
myUsername = "Valid"
myPassword = "ValidOnly"

# comment out the next lines, if the creds are hard-coded instead
def readCredsFromFile(filePathName):
    global myUsername, myPassword
    credFile = open(filePathName,"r")    # one per line
                                               
    myUsername = credFile.readline().rstrip('\n')
    myPassword = credFile.readline().rstrip('\n')

    credFile.close()

readCredsFromFile("..\creds\credsDSS.txt")
#print(myUsername)


### Authentication - Datascope Token Request

In [7]:
def requestToken(dssUsername, dssRassword) :
    requestUrl = reqStart + ".refinitiv.com/RestApi/v1/Authentication/RequestToken"

    requestHeaders={
        "Prefer":"respond-async",
        "Content-Type":"application/json"
        }

    requestBody={
        "Credentials": {
        "Username": myUsername,
        "Password": myPassword
    }
    }

    r1 = requests.post(requestUrl, json=requestBody,headers=requestHeaders)

    if r1.status_code == 200 :
        jsonResponse = json.loads(r1.text.encode('ascii', 'ignore'))
        token = jsonResponse["value"]
   #     print ('Authentication token (valid 24 hours):')
   #     print (token)
    else:
        print ('Replace myUserName and myPassword with valid credentials, then repeat the request')
        token = 'None'
    
    return token

tokenValid = requestToken(myUsername, myPassword)
print("Authentication token (valid 24 hours): "+tokenValid)

Authentication token (valid 24 hours): _JVeCpRtBMIgBCGQIGgvuxifEo7c3nDxxr85G3g6YjdFGqW3-LZqn_naPVZUiqpLW8sK7cqB_uMakOIoDj7RJuOcaZtjW5gzpk41IvxEAY3G3k3M-3pOoVYASIzTu0o-u2MtVUP4l8F3t-N_daUZS9CEsmdpw018moqwQbfCHAwFpC7DO4V_R17x-l3KJJ103SbCjhdWE5X5FOXWqRFQ5X7oQVFZhOkGQiWaMHpxk1MKjMIY6Wc0pYHEj82t3tPRUxoaHoktJJVqQY8xlj7QqId91QAs8fxUlE6VJRBnzHEQ


### Request all user packages
Select package that is of interest and remember it's packageId

In [37]:
def requestUserPackages():
    requestUrl = reqStart + '.refinitiv.com/RestApi/v1/StandardExtractions/UserPackages'

    requestHeaders={
        "Prefer":"respond-async",
        "Content-Type":"application/json",
        "Authorization": "token " + tokenValid
    }

    response = requests.get(requestUrl,headers=requestHeaders)

    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.request("GET", FILESET_ENDPOINT, data=payload, headers=headers, params=querystring)
    
    if response.status_code == 200:
        jsonFullResp = json.loads(response.text)        
        return jsonFullResp; 
    else:
        return '';
    
resp = requestUserPackages()
print(json.dumps(resp, indent=2));

packageId = ''
# from the response, we select the packageId for RRUS = Russian Sunctions
for i in range(len(resp['value'])): 
    if 'RRUS' in resp['value'][i]['PackageName']:
        packageId = resp['value'][i]['UserPackageId']
        print('Found: ',resp['value'][i]['PackageName']) 
        break;

if packageId == '':
    print("*** Attention!  no Russian sanctions found- user id is probably not permissioned!")

{
  "@odata.context": "https://selectapi.datascope.refinitiv.com/RestApi/v1/$metadata#UserPackages",
  "value": [
    {
      "UserPackageId": "0x04f21a8d13459cb1",
      "PackageId": "0x04f21a8d13459cb1",
      "PackageName": "ADC - NASD Alternative Display Facility for NYSE/AMEX Issues",
      "SubscriptionId": "0x0400dc1d24a00cb4",
      "SubscriptionName": "TRTH Venue by Day"
    },
    {
      "UserPackageId": "0x04f21a8d13759cb1",
      "PackageId": "0x04f21a8d13759cb1",
      "PackageName": "ADE - ATHENS DERIVATIVES EXCHANGE",
      "SubscriptionId": "0x0400dc1d24a00cb4",
      "SubscriptionName": "TRTH Venue by Day"
    },
    {
      "UserPackageId": "0x04f21a8d13859cb1",
      "PackageId": "0x04f21a8d13859cb1",
      "PackageName": "ADF - NASD Alternative Display Facility for Nasdaq Large Cap",
      "SubscriptionId": "0x0400dc1d24a00cb4",
      "SubscriptionName": "TRTH Venue by Day"
    },
    {
      "UserPackageId": "0x04f21a8d13a59cb1",
      "PackageId": "0x04f21a8d13a5

### Request packages deliveries that are available for the package
Select package delivery that is of interest and remember it's packageDeliveryId

In [38]:
def requestPackagesDeliveries():
    requestUrl = reqStart + '.refinitiv.com/RestApi/v1/StandardExtractions/UserPackageDeliveryGetUserPackageDeliveriesByPackageId(PackageId=\'' + packageId+'\')'

    requestHeaders={
        "Prefer":"respond-async",
        "Content-Type":"application/json",
        "Authorization": "token " + tokenValid
    }

    response = requests.get(requestUrl,headers=requestHeaders)

    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
                accessToken = getToken();     # token refresh on token expired
                headers['Authorization'] = "Bearer " + accessToken
                response = requests.request("GET", FILESET_ENDPOINT, data=payload, headers=headers, params=querystring)
    
    if response.status_code == 200:
        jsonFullResp = json.loads(response.text)        
        return jsonFullResp; 
    else:
        return '';
    
resp = requestPackagesDeliveries()
print(json.dumps(resp, indent=2));

# form the response, we select 1st packageDeliveryId '0'
packageDeliveryId = resp['value'][0]['PackageDeliveryId']
fileName = resp['value'][0]['Name']
print('Selected packageDeliveryId= ', packageDeliveryId, 'fileName= ',fileName)

{
  "@odata.context": "https://selectapi.datascope.refinitiv.com/RestApi/v1/$metadata#UserPackageDeliveries",
  "value": [
    {
      "PackageDeliveryId": "0x07ee43f0d75da96c",
      "UserPackageId": "0x06559b6f5b6b2f96",
      "SubscriptionId": "0x06516d2d08ab3036",
      "Name": "UK_Russian_Ukrainian_Sanctions_OCP_20220310.xlsx",
      "CreateDateTime": "2022-03-10T06:19:17.750Z",
      "ReleaseDateTime": "2022-03-10T12:00:00.000Z",
      "FileSizeBytes": 596097,
      "Frequency": "Weekly",
      "ContentMd5": ""
    },
    {
      "PackageDeliveryId": "0x07ee255cf77da921",
      "UserPackageId": "0x06559b6f5b6b2f96",
      "SubscriptionId": "0x06516d2d08ab3036",
      "Name": "EURussianSanctions_OCP_20220310.xlsx",
      "CreateDateTime": "2022-03-10T06:19:12.870Z",
      "ReleaseDateTime": "2022-03-10T12:00:00.000Z",
      "FileSizeBytes": 825786,
      "Frequency": "Weekly",
      "ContentMd5": ""
    },
    {
      "PackageDeliveryId": "0x07ee449c169da972",
      "UserPackageId

### Retrieve Sanction File and Save- Define
* Get the extraction results, using packageDeliveryId
* Save file

In [46]:
def retrieveFile(token, packageDeliveryId, fileName):
    requestUrl = reqStart + '.refinitiv.com/RestApi/v1/StandardExtractions/UserPackageDeliveries(\''+packageDeliveryId+'\')/$value'

    requestHeaders={
            "Prefer":"respond-async",
            "Content-Type":"application/json",
            "Accept-Encoding":"gzip",
            "Authorization": "token " + token
    }

    r = requests.get(requestUrl,headers=requestHeaders,stream=True)    
    r.raw.decode_content = False
    
    fileName = filePath + fileName
    print ('Saving data to file:' + fileName )
   
    chunk_size = 1024
    rr = r.raw
    with open(fileName, 'wb') as fd:
        shutil.copyfileobj(rr, fd, chunk_size)
    fd.close

    print ('Finished saving data to file: ' + fileName + '\n')
    
retrieveFile(tokenValid, packageDeliveryId, fileName)


Saving data to file:.\downloads\USRussianSanctions_OCP_20220214.xlsx
Finished saving data to file: .\downloads\USRussianSanctions_OCP_20220214.xlsx



## Retrieve all Sanction Files in Package 

In [45]:
for i in range(len(resp['value'])): 
    packageDeliveryId = resp['value'][i]['PackageDeliveryId']
    fileName = resp['value'][i]['Name']
    print('Selected packageDeliveryId= ', packageDeliveryId, 'fileName= ',fileName)
    retrieveFile(tokenValid, packageDeliveryId, fileName)

Selected packageDeliveryId=  0x07ee43f0d75da96c fileName=  UK_Russian_Ukrainian_Sanctions_OCP_20220310.xlsx
Saving data to file:.\downloads\UK_Russian_Ukrainian_Sanctions_OCP_20220310.xlsx
Finished saving data to file: .\downloads\UK_Russian_Ukrainian_Sanctions_OCP_20220310.xlsx

Selected packageDeliveryId=  0x07ee255cf77da921 fileName=  EURussianSanctions_OCP_20220310.xlsx
Saving data to file:.\downloads\EURussianSanctions_OCP_20220310.xlsx
Finished saving data to file: .\downloads\EURussianSanctions_OCP_20220310.xlsx

Selected packageDeliveryId=  0x07ee449c169da972 fileName=  USRussianSanctions_OCP_20220310.xlsx
Saving data to file:.\downloads\USRussianSanctions_OCP_20220310.xlsx
Finished saving data to file: .\downloads\USRussianSanctions_OCP_20220310.xlsx

Selected packageDeliveryId=  0x07edda67815da8bc fileName=  UK_Russian_Ukrainian_Sanctions_OCP_20220309.xlsx
Saving data to file:.\downloads\UK_Russian_Ukrainian_Sanctions_OCP_20220309.xlsx
Finished saving data to file: .\download